# ChemProp Pre-trained Molecular Features

**Strategy**: Use ChemProp's RDKit 2D features (217 dims) and Morgan fingerprints (2048 dims) as molecular representations instead of Spange/DRFP.

**Hypothesis**: Pre-trained molecular representations may capture structural information better than hand-crafted features, potentially changing the CV-LB relationship.

**Model**: CatBoost+XGBoost ensemble (best performing tabular model)

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# ChemProp feature extraction
from chemprop.featurizers import RDKit2DFeaturizer, MorganBinaryFeaturizer
from rdkit import Chem
import tqdm

# Load SMILES lookup
DATA_PATH = '/home/data'
smiles_df = pd.read_csv(f'{DATA_PATH}/smiles_lookup.csv')
smiles_dict = dict(zip(smiles_df['SOLVENT NAME'], smiles_df['solvent smiles']))

print(f'Loaded {len(smiles_dict)} solvents')
print('Sample:', list(smiles_dict.items())[:3])

Loaded 26 solvents
Sample: [('Cyclohexane', 'C1CCCCC1'), ('Ethyl Acetate', 'O=C(OCC)C'), ('Acetic Acid', 'CC(=O)O')]


In [3]:
# Extract ChemProp features for all solvents
rdkit_featurizer = RDKit2DFeaturizer()
morgan_featurizer = MorganBinaryFeaturizer(radius=2, length=2048)

rdkit_features = {}
morgan_features = {}

for solvent, smiles in smiles_dict.items():
    # Handle mixture solvents (e.g., 'Water.Acetonitrile')
    if '.' in smiles:
        # Split and average features
        parts = smiles.split('.')
        rdkit_parts = []
        morgan_parts = []
        for part in parts:
            mol = Chem.MolFromSmiles(part)
            if mol is not None:
                rdkit_parts.append(rdkit_featurizer(mol))
                morgan_parts.append(morgan_featurizer(mol))
        if rdkit_parts:
            rdkit_features[solvent] = np.mean(rdkit_parts, axis=0)
            morgan_features[solvent] = np.mean(morgan_parts, axis=0)
    else:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            rdkit_features[solvent] = rdkit_featurizer(mol)
            morgan_features[solvent] = morgan_featurizer(mol)

print(f'RDKit features: {len(rdkit_features)} solvents, {len(list(rdkit_features.values())[0])} dims')
print(f'Morgan features: {len(morgan_features)} solvents, {len(list(morgan_features.values())[0])} dims')

The RDKit 2D features can deviate signifcantly from a normal distribution. Consider manually scaling them using an appropriate scaler before creating datapoints, rather than using the scikit-learn `StandardScaler` (the default in Chemprop).


RDKit features: 26 solvents, 217 dims
Morgan features: 26 solvents, 2048 dims


In [4]:
# Create feature DataFrames
RDKIT_DF = pd.DataFrame(rdkit_features).T
MORGAN_DF = pd.DataFrame(morgan_features).T

# Handle NaN/Inf in RDKit features
RDKIT_DF = RDKIT_DF.replace([np.inf, -np.inf], np.nan)
RDKIT_DF = RDKIT_DF.fillna(0)

# Filter Morgan to non-zero variance columns
morgan_variance = MORGAN_DF.var()
nonzero_morgan_cols = morgan_variance[morgan_variance > 0].index.tolist()
MORGAN_FILTERED = MORGAN_DF[nonzero_morgan_cols]

print(f'RDKit features: {RDKIT_DF.shape}')
print(f'Morgan filtered: {MORGAN_FILTERED.shape}')

RDKit features: (26, 217)
Morgan filtered: (26, 110)


In [5]:
# Data loading functions
INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [6]:
# Load existing feature lookups for comparison
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [7]:
# ChemProp Featurizer - uses RDKit 2D + Morgan fingerprints
class ChemPropFeaturizer:
    def __init__(self, mixed=False, use_pca=True, pca_components=20):
        self.mixed = mixed
        self.rdkit_df = RDKIT_DF
        self.morgan_df = MORGAN_FILTERED
        self.use_pca = use_pca
        self.pca_components = pca_components
        
        # Combine RDKit + Morgan features
        # Use PCA to reduce dimensionality of Morgan fingerprints
        if use_pca:
            n_components = min(pca_components, len(MORGAN_FILTERED.columns), len(MORGAN_FILTERED) - 1)
            self.morgan_pca = PCA(n_components=n_components)
            morgan_pca_feats = self.morgan_pca.fit_transform(MORGAN_FILTERED.values)
            self.morgan_pca_df = pd.DataFrame(morgan_pca_feats, index=MORGAN_FILTERED.index)
            self.feats_dim = 2 + 3 + self.rdkit_df.shape[1] + self.morgan_pca_df.shape[1]
        else:
            self.feats_dim = 2 + 3 + self.rdkit_df.shape[1] + self.morgan_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_rdkit = self.rdkit_df.loc[X["SOLVENT A NAME"]].values
            B_rdkit = self.rdkit_df.loc[X["SOLVENT B NAME"]].values
            if self.use_pca:
                A_morgan = self.morgan_pca_df.loc[X["SOLVENT A NAME"]].values
                B_morgan = self.morgan_pca_df.loc[X["SOLVENT B NAME"]].values
            else:
                A_morgan = self.morgan_df.loc[X["SOLVENT A NAME"]].values
                B_morgan = self.morgan_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_rdkit = B_rdkit * (1 - (1-pct)) + A_rdkit * (1-pct)
                X_morgan = B_morgan * (1 - (1-pct)) + A_morgan * (1-pct)
            else:
                X_rdkit = A_rdkit * (1 - pct) + B_rdkit * pct
                X_morgan = A_morgan * (1 - pct) + B_morgan * pct
        else:
            X_rdkit = self.rdkit_df.loc[X["SOLVENT NAME"]].values
            if self.use_pca:
                X_morgan = self.morgan_pca_df.loc[X["SOLVENT NAME"]].values
            else:
                X_morgan = self.morgan_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_rdkit, X_morgan])
    
    def featurize_torch(self, X, flip=False):
        return torch.tensor(self.featurize(X, flip))

print(f'ChemProp feature dimension: {ChemPropFeaturizer().feats_dim}')

ChemProp feature dimension: 242


In [8]:
# CatBoost + XGBoost Ensemble with ChemProp features
import catboost as cb
import xgboost as xgb

class ChemPropCatXGBEnsemble:
    def __init__(self, data='single'):
        self.data = data
        self.mixed = (data == 'full')
        self.featurizer = ChemPropFeaturizer(mixed=self.mixed, use_pca=True, pca_components=20)
        self.scalers = [StandardScaler() for _ in range(3)]
        self.catboost_models = []
        self.xgb_models = []
        self.cat_weight = 0.5
        self.xgb_weight = 0.5
        
    def train_model(self, X, Y):
        X_feats = self.featurizer.featurize(X)
        Y_vals = Y.values
        
        self.catboost_models = []
        self.xgb_models = []
        
        for i in range(3):
            y_scaled = self.scalers[i].fit_transform(Y_vals[:, i:i+1]).ravel()
            
            # CatBoost
            cat_model = cb.CatBoostRegressor(
                iterations=500,
                learning_rate=0.05,
                depth=6,
                l2_leaf_reg=3,
                random_seed=42,
                verbose=False
            )
            cat_model.fit(X_feats, y_scaled)
            self.catboost_models.append(cat_model)
            
            # XGBoost
            xgb_model = xgb.XGBRegressor(
                n_estimators=500,
                learning_rate=0.05,
                max_depth=6,
                reg_lambda=3,
                random_state=42,
                verbosity=0
            )
            xgb_model.fit(X_feats, y_scaled)
            self.xgb_models.append(xgb_model)
    
    def predict(self, X):
        X_feats = self.featurizer.featurize(X)
        preds = []
        
        for i in range(3):
            cat_pred = self.catboost_models[i].predict(X_feats)
            xgb_pred = self.xgb_models[i].predict(X_feats)
            
            # Ensemble
            pred_scaled = self.cat_weight * cat_pred + self.xgb_weight * xgb_pred
            pred = self.scalers[i].inverse_transform(pred_scaled.reshape(-1, 1)).ravel()
            preds.append(pred)
        
        return torch.tensor(np.stack(preds, axis=1))

print('ChemPropCatXGBEnsemble defined')

ChemPropCatXGBEnsemble defined


In [9]:
# Cross-validation on single solvent data
import tqdm

X, Y = load_data("single_solvent")
print(f'Single solvent data: {len(X)} samples, {len(X["SOLVENT NAME"].unique())} solvents')

all_mse = []
for fold_idx, split in tqdm.tqdm(enumerate(generate_leave_one_out_splits(X, Y))):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = ChemPropCatXGBEnsemble(data='single')
    model.train_model(train_X, train_Y)
    
    preds = model.predict(test_X).numpy()
    targets = test_Y.values
    
    mse = np.mean((preds - targets) ** 2)
    all_mse.append(mse)

single_mse = np.mean(all_mse)
print(f'\nSingle Solvent MSE: {single_mse:.6f}')

Single solvent data: 656 samples, 24 solvents


0it [00:00, ?it/s]

1it [00:02,  2.23s/it]

2it [00:04,  2.19s/it]

3it [00:06,  2.16s/it]

...

22it [00:47,  2.14s/it]

23it [00:49,  2.16s/it]

24it [00:52,  2.16s/it]

24it [00:52,  2.17s/it]


Single Solvent MSE: 0.015480


In [ ]:
# Cross-validation on full data
X_full, Y_full = load_data("full")
print(f'Full data: {len(X_full)} samples')

all_mse_full = []
for fold_idx, split in tqdm.tqdm(enumerate(generate_leave_one_ramp_out_splits(X_full, Y_full))):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = ChemPropCatXGBEnsemble(data='full')
    model.train_model(train_X, train_Y)
    
    preds = model.predict(test_X).numpy()
    targets = test_Y.values
    
    mse = np.mean((preds - targets) ** 2)
    all_mse_full.append(mse)

full_mse = np.mean(all_mse_full)
print(f'\nFull Data MSE: {full_mse:.6f}')

In [ ]:
# Calculate overall CV score
n_single = 656
n_full = 2496
overall_mse = (single_mse * n_single + full_mse * n_full) / (n_single + n_full)

print(f'\n=== ChemProp Features Results ===')
print(f'Single Solvent MSE: {single_mse:.6f}')
print(f'Full Data MSE: {full_mse:.6f}')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest baseline (CatBoost+XGBoost with Spange/DRFP): 0.008092')
print(f'Difference: {(overall_mse - 0.008092) / 0.008092 * 100:.2f}%')

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = ChemPropCatXGBEnsemble(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = ChemPropCatXGBEnsemble(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################